Per the section on turn order at https://gamefaqs.gamespot.com/nes/522595-final-fantasy/faqs/57009, turn order is scheduled by listing the ememies followed by the players, and swapping random index pairs.

In [63]:
from random import randint

def make_sched():
    sched = [f'e{i}' for i in range(9)] + ['p']*4
    for _ in range(17):
        a, b = randint(0,12), randint(0,12)
        va, vb = sched[a], sched[b]
        sched[a], sched[b] = vb, va
    return sched
make_sched()

['e4', 'e2', 'e0', 'e3', 'e8', 'e7', 'e6', 'p', 'e5', 'p', 'e1', 'p', 'p']

The final schedule is determined by skipping any enemies or players who are stunned, petrified, or dead

In [50]:
def make_fsched(s, elist=None, np=4):
    """
    :param s: shuffled order of enemies and player characters
    :param elist: list of enemy ids that are eligible for an action. None if they all are eligible.
    :param np: number of players that are eligible for an action
    """
    # create final schedule
    ne = 9 if elist is None else len(elist)
    fs = []
    e = 0
    p = 0
    for item in s:
        if item == 'p' and p < np:
            fs.append(item)
            p += 1
        elif (elist is None) or (item in elist):
            fs.append(item)
    assert len(fs)==ne+np, fs
    return fs

stest = make_sched()
print(stest)
print(make_fsched(stest))
make_fsched(stest, elist=['e0'], np=2)

['e5', 'e0', 'e3', 'p', 'e4', 'e6', 'p', 'p', 'e1', 'e8', 'p', 'e2', 'e7']
['e5', 'e0', 'e3', 'p', 'e4', 'e6', 'p', 'p', 'e1', 'e8', 'p', 'e2', 'e7']


['e0', 'p', 'p']

In [47]:
slist = [make_sched() for _ in range(100)]

Original code did not consider specific enemy ids, resulting in odd results.  E.g. if any enemy was is a slot, it was assigned to the schedule even if that specific id would have been ineligible.

Now we assume that ids `e{0}..e{num enemies-1}` are eligible, and the rest are not.

In [62]:
np = 4
for ne in range(1,10):
    # make final schedules for number of enemies
    fslist = [make_fsched(s,elist=[f'e{i}' for i in range(ne)]) for s in slist]
    
    # for each slot, get the percentage of player values
    slot_pct = {}
    for slot in range(np+ne):
        slots = [fs[slot] for fs in fslist]
        slot_pct[slot] = slots.count('p')/len(slots)
    
    print(f'{ne=}:\n\t{slot_pct}')
    
    # compare to expected percentage of player values
    expected = np/(np+ne)
    disadv = {k:round(act-expected,2) for k,act in slot_pct.items()}
    print(f'\t{disadv}')
    

ne=1:
	{0: 0.72, 1: 0.78, 2: 0.82, 3: 0.85, 4: 0.83}
	{0: -0.08, 1: -0.02, 2: 0.02, 3: 0.05, 4: 0.03}
ne=2:
	{0: 0.61, 1: 0.59, 2: 0.74, 3: 0.64, 4: 0.75, 5: 0.67}
	{0: -0.06, 1: -0.08, 2: 0.07, 3: -0.03, 4: 0.08, 5: 0.0}
ne=3:
	{0: 0.52, 1: 0.51, 2: 0.58, 3: 0.61, 4: 0.52, 5: 0.65, 6: 0.61}
	{0: -0.05, 1: -0.06, 2: 0.01, 3: 0.04, 4: -0.05, 5: 0.08, 6: 0.04}
ne=4:
	{0: 0.46, 1: 0.41, 2: 0.49, 3: 0.46, 4: 0.57, 5: 0.53, 6: 0.54, 7: 0.54}
	{0: -0.04, 1: -0.09, 2: -0.01, 3: -0.04, 4: 0.07, 5: 0.03, 6: 0.04, 7: 0.04}
ne=5:
	{0: 0.41, 1: 0.4, 2: 0.41, 3: 0.35, 4: 0.5, 5: 0.5, 6: 0.48, 7: 0.46, 8: 0.49}
	{0: -0.03, 1: -0.04, 2: -0.03, 3: -0.09, 4: 0.06, 5: 0.06, 6: 0.04, 7: 0.02, 8: 0.05}
ne=6:
	{0: 0.39, 1: 0.33, 2: 0.42, 3: 0.27, 4: 0.38, 5: 0.43, 6: 0.5, 7: 0.43, 8: 0.38, 9: 0.47}
	{0: -0.01, 1: -0.07, 2: 0.02, 3: -0.13, 4: -0.02, 5: 0.03, 6: 0.1, 7: 0.03, 8: -0.02, 9: 0.07}
ne=7:
	{0: 0.36, 1: 0.33, 2: 0.36, 3: 0.28, 4: 0.31, 5: 0.32, 6: 0.42, 7: 0.44, 8: 0.39, 9: 0.33, 10: 0.46}
	{0: -0